In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("spam.csv",encoding="latin-1")

In [ ]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [ ]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)

In [ ]:
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df["target"]=encoder.fit_transform(df["target"])

In [ ]:
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.isnull().sum()

,0
target,0
text,0


In [ ]:
df["target"].unique()

array([0, 1])

In [ ]:
df["target"].value_counts()

,count
target,
0,4825
1,747


In [ ]:
df.duplicated().sum()

403

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps=PorterStemmer()
def transform_text(text):
  y=[]
  text=text.lower()
  text=nltk.word_tokenize(text)
  for i in text:
    if i.isalnum():
      y.append(i)
  text=y[:]
  y.clear()
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)
  text=y[:]
  y.clear()
  for i in text:
    y.append(ps.stem(i))
  return " ".join(y)



In [ ]:
transform_text("Hi my name is abhishek and i m eating panner!")

'hi name abhishek eat panner'

In [ ]:
df["text"]=df["text"].apply(transform_text)

In [ ]:
df.head()

,target,text
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri 2 wkli comp win fa cup final tkt 21...
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid=TfidfVectorizer()
X=tfid.fit_transform(df["text"]).toarray()
Y=df["target"]

In [ ]:
X.shape,Y.shape

((5169, 6708), (5169,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((4135, 6708), (1034, 6708), (4135,), (1034,))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
model=Sequential()
model.add(Dense(32,activation="relu",input_shape=(X_train[1].shape)))
model.add(Dropout(0.2))
model.add(Dense(16,activation="linear"))
model.add(Dropout(0.3))
model.add(Dense(1,activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8414 - loss: 0.5208 - val_accuracy: 0.8985 - val_loss: 0.2047
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9592 - loss: 0.1288 - val_accuracy: 0.9720 - val_loss: 0.0894
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9873 - loss: 0.0465 - val_accuracy: 0.9749 - val_loss: 0.0829
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9936 - loss: 0.0238 - val_accuracy: 0.9758 - val_loss: 0.0829
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9968 - loss: 0.0122 - val_accuracy: 0.9758 - val_loss: 0.0852
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9973 - loss: 0.0097 - val_accuracy: 0.9749 - val_loss: 0.0909
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 0.9749 - val_loss: 0.0938
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9994 - loss: 0.0034 - val_accuracy: 

In [ ]:
X_test = np.nan_to_num(X_test)
X_test.shape,Y_test.shape

((1034, 6708), (1034,))

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9759 - loss: 0.1027
Loss: 0.10749320685863495, Accuracy: 0.9758220314979553


In [ ]:
def process_input(text):
  y=transform_text(text)
  y=tfid.transform([y]).toarray()
  return y

In [ ]:
def predict(text):
  y=process_input(text)
  y_pred=model.predict(y)
  print(y_pred)
  if y_pred>0.5:
     print("spam")
  else:
     print("ham")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.00309809]]
ham


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train,Y_train)

MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
nb_pred=nb.predict(X_test)

In [ ]:
accuracy=accuracy_score(Y_test,nb_pred)

In [ ]:
accuracy

0.9613152804642167

In [ ]:
def predict_nb(text):
  y=process_input(text)
  y_pred=nb.predict(y)
  print(y_pred)
  if y_pred==1:
     print("spam")
  else:
     print("ham")

In [ ]:
predict_nb("""Subject: 📰 Your Weekly Tech News Roundup
From: "Tech Digest" <newsletter@techdigest.com>
Message:

Hi [Your Name],

Here are the top tech stories this week:

Apple announces new AI-powered MacBook.
Google updates its search algorithm.
Python 3.12: What’s new?
👉 Read More

You’re receiving this because you subscribed to our newsletter. Want to unsubscribe? Click here.

Regards,
Tech Digest Team""")

[0]
ham


In [ ]:
predict("""Subject: 📰 Your Weekly Tech News Roundup
From: "Tech Digest" <newsletter@techdigest.com>
Message:

Hi [Your Name],

Here are the top tech stories this week:

Apple announces new AI-powered MacBook.
Google updates its search algorithm.
Python 3.12: What’s new?
👉 Read More

You’re receiving this because you subscribed to our newsletter. Want to unsubscribe? Click here.

Regards,
Tech Digest Team""",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
[[0.62234974]]
spam
